<a href="https://colab.research.google.com/github/AbdullahMakhdoom/Reverse_Image_Search/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Feature Vectors 

  The feature vectors (also called embeddings or bottleneck features) are essentially a collection of a few thousand floating-point values. Going through the convolution and pooling layers in a CNN is basically an act of reduction, to filter the information contained in the image to its most important and salient constituents, which in turn form the bottleneck features. Training the CNN molds these values in such a way that items (in our case: images) belonging to the same class have small Euclidean distance between them (or simply the square root of the sum of squares of the difference between corresponding values) and items from different classes are separated by larger distances. This is an important property which we will use in this project to develop an Image Retrival Enginer like Google's Image Search.

**Objective** : In this notebook, we will extract 2 sets of feature vectors of Caltech101 Dataset using ResNet50 as deep architecture. One will be extracted using Resnet50 pre-trained on ImageNet. The other will be fine-tuned over Caltech101.

**Dataset** : Caltech 101 dataset has a size of 131 MB with approximately 9000 
images in 101 categories (about 40 to 800 images per category). 

*Note* : *A 102nd category called "BACKGROUND_Google" consisting of random images is also present in this dataset,which needs to be deleted.*

Download the Caltech101 dataset.

In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp --output caltech101.tar.gz
!tar -xvzf caltech101.tar.gz
!mkdir datasets
!mv 101_ObjectCategories datasets/caltech101
!rm -rf datasets/caltech101/BACKGROUND_Google

**Import Necessary Packages**

In [ ]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D


A Utility function that allows us to choose a pretrained model architecture, pre-trained on ImageNet, with all the necessary details for experimentation.

In [ ]:
def model_picker(name):
    if (name == 'vgg16'):
        model = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'vgg19'):
        model = VGG19(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'mobilenet'):
        model = MobileNet(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3),
                          pooling='max',
                          depth_multiplier=1,
                          alpha=1)
    elif (name == 'inception'):
        model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=(224, 224, 3),
                            pooling='max')
    elif (name == 'resnet'):
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')
    elif (name == 'xception'):
        model = Xception(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    else:
        print("Specified model not available")
    return model

Picking 'ResNet50' as our model architecture. 

In [ ]:
model_architecture = 'resnet'
model = model_picker(model_architecture)

94773248/94765736 [==============================] - 1s 0us/step


Defining a function to extract image features from an image and a model.

In [ ]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                         target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

Defining a helper function to recursively get all image files under a root directory.


In [ ]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

Running the extraction over the entire Caltech101 dataset and timing it using tqdm package.

In [ ]:
# path to the your datasets
root_dir = 'datasets/caltech101'
filenames = sorted(get_file_list(root_dir))

feature_list = []
for i in notebook.tqdm(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))

In [ ]:
# make folder named 'data'
!mkdir data

Saving the features and filenames for later use.

Utilizing parallel processing of GPUs will speed up the feature extraction step.
(Keeping the batch_size = 64)

In [ ]:
batch_size = 64
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory(root_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(generator.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

start_time = time.time()
feature_list = []
feature_list = model.predict_generator(generator, num_epochs)
end_time = time.time()

Found 8677 images belonging to 101 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Utilizing Colab's GPU fastened the feature extraction process approximately 3 times (4080(sec)/ 1331(sec) = 3.065)

In [ ]:
pickle.dump(feature_list, open('data/features-caltech101-resnet.pickle', 'wb'))
pickle.dump(filenames, open('data/filenames-caltech101.pickle', 'wb'))

**Fine-tuned Model**

In [ ]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 101
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size = 64,
                                                    shuffle = True,
                                                    seed = 42,
                                                    class_mode= 'categorical')


In [ ]:
def model_maker():
  base_model = ResNet50(include_top = False,
                        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
  for layer in base_model.layers[:]:
    layer.trainable = False
  input = Input(shape = (IMG_WIDTH, IMG_HEIGHT, 3))
  custom_model = base_model(input)
  custom_model = GlobalAveragePooling2D()(custom_model)
  custom_model = Dense(64, activation='relu')(custom_model)
  custom_model = Dropout(0.5)(custom_model)
  predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
  return Model(inputs=input, outputs=predictions)



In [ ]:
model_finetuned = model_maker()
model_finetuned.compile(loss = 'categorical_crossentropy',
                        optimizer=tensorflow.keras.optimizers.Adam(0.001),
                        metrics=['acc'])
model_finetuned.fit(
    train_generator,
    steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs = 10)


Epoch 1/10
136/136 [==============================] - 1371s 10s/step - loss: 3.4368 - acc: 0.2723
Epoch 2/10
136/136 [==============================] - 1391s 10s/step - loss: 1.7939 - acc: 0.5703
Epoch 3/10
136/136 [==============================] - 1381s 10s/step - loss: 1.3258 - acc: 0.6644
Epoch 4/10
136/136 [==============================] - 1389s 10s/step - loss: 1.1050 - acc: 0.7030
Epoch 5/10
136/136 [==============================] - 1388s 10s/step - loss: 0.9940 - acc: 0.7311
Epoch 6/10
136/136 [==============================] - 1387s 10s/step - loss: 0.8527 - acc: 0.7579
Epoch 7/10
136/136 [==============================] - 1386s 10s/step - loss: 0.8143 - acc: 0.7631
Epoch 8/10
136/136 [==============================] - 1381s 10s/step - loss: 0.7317 - acc: 0.7878
Epoch 9/10
121/136 [=========================>....] - ETA: 2:34 - loss: 0.7600 - acc: 0.7750

In [ ]:
model_finetuned.save('./data/model-finetuned.h5')

In [ ]:
start_time = time.time()
feature_list_finetuned = []
feature_list_finetuned = model_finetuned.predict_generator(generator, num_epochs)
end_time = time.time()

for i, features_finetuned in enumerate(feature_list_finetuned):
    feature_list_finetuned[i] = features_finetuned / norm(features_finetuned)

feature_list = feature_list_finetuned.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

In [ ]:
pickle.dump(
    feature_list,
    open('./data/features-caltech101-resnet-finetuned.pickle', 'wb'))

Saving '.pickle' files to Google Drive.

In [ ]:
# Mount Google Drive
from google.colab import driv
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Make new directory in MyDrive
!mkdir /content/drive/MyDrive/Caltech101-features

In [ ]:
!mv ./data/features-caltech101-resnet-finetuned.pickle /content/drive/MyDrive/Caltech101-features/

In [ ]:
!mv ./data/features-caltech101-resnet.pickle /content/drive/MyDrive/Caltech101-features/

In [ ]:
!mv ./data/filenames-caltech101.pickle /content/drive/MyDrive/Caltech101-features/